In [1]:
import os
from src.zmongo_retriever import ZMongoRetriever
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.llamacpp import LlamaCpp
from langchain.chains import load_summarize_chain


Set your variables

In [6]:
model_path = model_path_1 = '/mnt/storage/models/dolphin-2.1-mistral-7B-GGUF/dolphin-2.1-mistral-7b.Q4_0.gguf'
 # Your .gguf file path
mongo_db_name = 'case_graph' # Your MongoDB database name
mongo_uri = 'mongodb://localhost:49999' # Your mongo_uri
this_collection_name = 'zcases'  # Your MongoDB collection
this_page_content_field = 'opinion'  # Specify the field to use as page_content
this_document_id = '65d9957e2051723e1bb6eec9'  # Example ObjectId('_id') value

Use ZMongoRetriever to split the text from the page_content_field into LangChain Documents

In [7]:
# larger values for chunk_size may solve problems with exceeding your token limit
retriever = ZMongoRetriever(mongo_uri=mongo_uri, chunk_size=1000, collection_name=this_collection_name, page_content_field=this_page_content_field)
documents_by_id = retriever.invoke(this_document_id, query_by_id=True)
print(documents_by_id)

[[<zmongo_retriever.Document object at 0x7f0e2fbf6290>, <zmongo_retriever.Document object at 0x7f0e2fbf4550>, <zmongo_retriever.Document object at 0x7f0e2fbf4220>, <zmongo_retriever.Document object at 0x7f0e2fbf5c60>, <zmongo_retriever.Document object at 0x7f0e2fbf4790>, <zmongo_retriever.Document object at 0x7f0e2fbf4970>, <zmongo_retriever.Document object at 0x7f0e2fbf62f0>, <zmongo_retriever.Document object at 0x7f0e2fbf4c40>, <zmongo_retriever.Document object at 0x7f0e2fbf5750>, <zmongo_retriever.Document object at 0x7f0e2fbf5300>, <zmongo_retriever.Document object at 0x7f0e2fbf5720>, <zmongo_retriever.Document object at 0x7f0e2fbf4190>, <zmongo_retriever.Document object at 0x7f0e2fbf5060>]]


Use LlamaCPP to summarize the page_content_field

In [9]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
llm = LlamaCpp(
    model_path=os.getenv('MODEL_PATH', '/mnt/storage/models/dolphin-2.1-mistral-7B-GGUF/dolphin-2.1-mistral-7b.Q4_0.gguf'),
    max_tokens=0,
    n_gpu_layers=-1,
    n_ctx=4096,
    n_batch=4096,
    verbose=True,
    f16_kv=True
)
stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary_chain_result = stuff_chain.invoke({'input_documents': documents_by_id[0]})
print(summary_chain_result)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /mnt/storage/models/dolphin-2.1-mistral-7B-GGUF/dolphin-2.1-mistral-7b.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ehartford_dolphin-2.1-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_lo

{'input_documents': [<zmongo_retriever.Document object at 0x7f0e2fbf6290>, <zmongo_retriever.Document object at 0x7f0e2fbf4550>, <zmongo_retriever.Document object at 0x7f0e2fbf4220>, <zmongo_retriever.Document object at 0x7f0e2fbf5c60>, <zmongo_retriever.Document object at 0x7f0e2fbf4790>, <zmongo_retriever.Document object at 0x7f0e2fbf4970>, <zmongo_retriever.Document object at 0x7f0e2fbf62f0>, <zmongo_retriever.Document object at 0x7f0e2fbf4c40>, <zmongo_retriever.Document object at 0x7f0e2fbf5750>, <zmongo_retriever.Document object at 0x7f0e2fbf5300>, <zmongo_retriever.Document object at 0x7f0e2fbf5720>, <zmongo_retriever.Document object at 0x7f0e2fbf4190>, <zmongo_retriever.Document object at 0x7f0e2fbf5060>], 'output_text': "1. U.S. Bank failed to reestablish a lost note, so the final judgment of foreclosure is reversed.\n  2. U.S. Bank's witness, Vonterro White, did not establish that the original lender was entitled to enforce the note when it was lost or whether the note was lo